In [1]:
# Install and load the required package if not already installed
if (!requireNamespace("readr", quietly = TRUE)) {
  install.packages("readr")
}
library(readr)

# Define the file paths for all CSV files
file_paths <- c("trips_import_2020-06.csv")

# Initialize an empty dataframe
trips <- data.frame()

# Loop through each file, read it, and append to the trips dataframe
for (file_path in file_paths) {
  temp_df <- read_csv(file_path)
  trips <- rbind(trips, temp_df)
}

# View the first few rows of the combined dataframe
head(trips)

Rows: 345277 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (2): store_and_fwd_flag, pickup_weekday
dbl  (19): VendorID, passenger_count, trip_distance, RatecodeID, PULocationI...
dttm  (2): tpep_pickup_datetime, tpep_dropoff_datetime
date  (4): file_date, eom_plus_1, pickup_date, dropoff_date
time  (2): pickup_time, dropoff_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,⋯,pickup_date,pickup_time,dropoff_date,dropoff_time,trip_duration,pickup_hour,pickup_weekday,PULocationID_encoded,DOLocationID_encoded,VendorID_encoded
<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<date>,<time>,<date>,<time>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,2020-06-01 00:31:23,2020-06-01 00:49:58,1,3.60,1,N,140,68,1,⋯,2020-06-01,00:31:23,2020-06-01,00:49:58,1115,0,Monday,15.97617,13.80704,15.29467
1,2020-06-01 00:42:50,2020-06-01 01:04:33,1,5.60,1,N,79,226,1,⋯,2020-06-01,00:42:50,2020-06-01,01:04:33,1303,0,Monday,15.39873,26.66805,15.29467
1,2020-06-01 00:39:51,2020-06-01 00:49:09,1,2.30,1,N,238,116,2,⋯,2020-06-01,00:39:51,2020-06-01,00:49:09,558,0,Monday,13.68903,18.22794,15.29467
1,2020-06-01 00:56:13,2020-06-01 01:11:38,1,5.30,1,N,141,116,2,⋯,2020-06-01,00:56:13,2020-06-01,01:11:38,925,0,Monday,13.38711,18.22794,15.29467
1,2020-06-01 00:16:41,2020-06-01 00:29:30,1,4.40,1,N,186,75,1,⋯,2020-06-01,00:16:41,2020-06-01,00:29:30,769,0,Monday,15.24436,13.24517,15.29467
2,2020-06-01 00:06:29,2020-06-01 00:13:22,1,1.72,1,N,142,186,2,⋯,2020-06-01,00:06:29,2020-06-01,00:13:22,413,0,Monday,13.83188,12.93527,15.47736


In [4]:
# Step 1: Select only numeric columns
numerical_cols <- names(trips)[sapply(trips, is.numeric)]

# Step 2: Exclude 'total_amount' from the numeric columns
numerical_cols <- numerical_cols[numerical_cols != 'total_amount']

# Step 3: Calculate the correlation matrix, handling potential issues with constant columns
# First, remove columns with zero variance to avoid "the standard deviation is zero" issue
trips_numerical <- trips[, numerical_cols]
trips_numerical <- trips_numerical[, sapply(trips_numerical, sd) != 0]

# Calculate the correlation matrix
corr_matrix <- cor(trips_numerical, use = "complete.obs")

# Step 4: Identify features with correlation higher than the threshold (0.8)
correlated_features <- c()
threshold <- 0.8

for (i in 1:(ncol(corr_matrix) - 1)) {
  for (j in (i + 1):ncol(corr_matrix)) {
    if (!is.na(corr_matrix[i, j]) && abs(corr_matrix[i, j]) > threshold) {
      correlated_features <- c(correlated_features, colnames(corr_matrix)[i])
    }
  }
}

# Step 5: Print out the correlated features
cat("Correlated features (above", threshold, ") to drop:", correlated_features, "\n")

# Step 6: Drop the correlated features but keep 'total_amount'
trips <- trips[, !(names(trips) %in% correlated_features)]

# Step 7: Display remaining columns
cat("Remaining features after dropping correlated ones:", colnames(trips), "\n")


Correlated features (above 0.8 ) to drop: VendorID VendorID trip_distance fare_amount extra 
Remaining features after dropping correlated ones: tpep_pickup_datetime tpep_dropoff_datetime passenger_count RatecodeID store_and_fwd_flag PULocationID DOLocationID payment_type mta_tax tip_amount tolls_amount improvement_surcharge total_amount file_date eom_plus_1 pickup_date pickup_time dropoff_date dropoff_time trip_duration pickup_hour pickup_weekday PULocationID_encoded DOLocationID_encoded VendorID_encoded 


In [5]:
# Step 1: Identify columns with zero variance (only one unique value)
zero_variance_cols <- names(trips)[sapply(trips, function(x) length(unique(x)) <= 1)]

# Step 2: Print columns with zero variance
cat("Columns with zero variance:", zero_variance_cols, "\n")

# Step 3: Drop columns with zero variance
trips <- trips[, !(names(trips) %in% zero_variance_cols)]

# Step 4: Print the remaining columns
cat("Remaining columns:", colnames(trips), "\n")


Columns with zero variance: passenger_count mta_tax tolls_amount improvement_surcharge file_date eom_plus_1 
Remaining columns: tpep_pickup_datetime tpep_dropoff_datetime RatecodeID store_and_fwd_flag PULocationID DOLocationID payment_type tip_amount total_amount pickup_date pickup_time dropoff_date dropoff_time trip_duration pickup_hour pickup_weekday PULocationID_encoded DOLocationID_encoded VendorID_encoded 


In [6]:
# Create dummy variables for 'RatecodeID' column
trips <- cbind(trips, model.matrix(~ RatecodeID - 1, data = trips))

# Remove the original 'RatecodeID' column as it's now encoded
trips$RatecodeID <- NULL


In [8]:
# Set the seed for reproducibility
set.seed(123)

# Get the indices of the dataset
dt_index <- 1:nrow(trips)

# Sample 80% of the indices
smpl_index <- sample(dt_index, size = floor(0.8 * length(dt_index)), replace = FALSE)

# Create the training set using the sampled indices
dt_train <- trips[smpl_index, ]

# Create the test set by excluding the sampled indices
dt_test <- trips[-smpl_index, ]

# Reset the index for the training set (reset row names in R)
rownames(dt_train) <- NULL

# Print the lengths of the datasets
cat("Length of training set:", nrow(dt_train), "\n")
cat("Length of test set:", nrow(dt_test), "\n")


Length of training set: 276221 
Length of test set: 69056 


In [11]:
install.packages("car")
library(car)  # For vif() function



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


Loading required package: carData



In [12]:
# Replace zeros with a small constant (1e-5) in total_amount column
dt_train$total_amount <- ifelse(dt_train$total_amount == 0, 1e-5, dt_train$total_amount)


In [30]:
# Define the initial formula
formula <- "log(total_amount) ~
  log(tip_amount + 1) +
  PULocationID_encoded +
  DOLocationID_encoded +
  VendorID_encoded +
  factor(store_and_fwd_flag) +
  factor(payment_type)"



  #  I(trip_distance^2) +
  #      RatecodeID_1 +
  #          log(passenger_count + 1) +
      # RatecodeID_2 +
        #  RatecodeID_3 +
            # RatecodeID_4 +
                # RatecodeID_5 +
  # RatecodeID_6 +

  # pickup_weekday_Friday +
  # pickup_weekday_Monday +
  # pickup_weekday_Saturday +
  # pickup_weekday_Sunday +
  # pickup_weekday_Thursday +
  # pickup_weekday_Tuesday"

In [32]:
# Define function to calculate VIFs
calculate_vifs <- function(model) {
  vif_data <- vif(model)

  # If vif_data is a matrix, convert it to a data frame
  if (is.matrix(vif_data)) {
    vif_data <- as.data.frame(vif_data)
    vif_data$Variable <- rownames(vif_data)
    colnames(vif_data) <- c("VIF", "Variable")  # Rename the columns for clarity
  }

  return(vif_data)
}
# Function to calculate and check for high VIFs
check_high_vifs <- function(vif_data) {
  high_vif_columns <- vif_data[vif_data$VIF > 4 | is.na(vif_data$VIF), "Variable"]
  return(high_vif_columns)
}


In [33]:
# Iteratively remove variables based on VIFs and p-values
repeat {
  # Fit the model
  mdl <- lm(formula, data = dt_train)

  # Calculate VIFs
  vif_data <- calculate_vifs(mdl)

  # Check for high VIFs (>4 or NA)
  high_vif_columns <- check_high_vifs(vif_data)

  # If high VIFs exist, remove the first variable with high VIF
  if (length(high_vif_columns) > 0) {
    variable_to_remove <- high_vif_columns[1]
    print(paste("Removing variable with high VIF:", variable_to_remove))

    # Update the formula by removing the variable
    formula <- update(formula, paste(". ~ . -", variable_to_remove))

    next  # Recalculate the model and VIFs after the change
  }

  # Get the summary table
  summary_table <- summary(mdl)

  # Identify variables with p-value > 0.05
  high_p_columns <- names(summary_table$coefficients)[summary_table$coefficients[, 4] > 0.05]

  # If no high p-values, break the loop
  if (length(high_p_columns) == 0) {
    print("No variables with high p-values remain.")
    break
  }

  # If there are high p-values, remove the first variable with high p-value
  variable_to_remove <- high_p_columns[1]
  print(paste("Removing variable with high p-value:", variable_to_remove))

  # Update the formula by removing the variable
  formula <- update(formula, paste(". ~ . -", variable_to_remove))
}


[1] "No variables with high p-values remain."


In [34]:
# After the final model fitting, print the VIFs
mdl <- lm(formula, data = dt_train)
vif_data <- calculate_vifs(mdl)
print("\nFinal VIFs:")
print(vif_data)


[1] "\nFinal VIFs:"
                                VIF Variable       NA
log(tip_amount + 1)        3.340834        1 1.827795
PULocationID_encoded       1.521520        1 1.233499
DOLocationID_encoded       1.518328        1 1.232204
VendorID_encoded           1.028508        1 1.014154
factor(store_and_fwd_flag) 1.013617        1 1.006785
factor(payment_type)       3.392878        4 1.164987
                                                   NA
log(tip_amount + 1)               log(tip_amount + 1)
PULocationID_encoded             PULocationID_encoded
DOLocationID_encoded             DOLocationID_encoded
VendorID_encoded                     VendorID_encoded
factor(store_and_fwd_flag) factor(store_and_fwd_flag)
factor(payment_type)             factor(payment_type)


In [35]:
# Print the final formula and summary
print("Final formula:")
print(formula)
print(summary(mdl))

[1] "Final formula:"
[1] "log(total_amount) ~ \n  log(tip_amount + 1) + \n  PULocationID_encoded + \n  DOLocationID_encoded + \n  VendorID_encoded + \n  factor(store_and_fwd_flag) + \n  factor(payment_type)"

Call:
lm(formula = formula, data = dt_train)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.77528 -0.21228 -0.01473  0.20757  1.37036 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  3.3470509  0.1089778  30.713  < 2e-16 ***
log(tip_amount + 1)          0.2658690  0.0018173 146.298  < 2e-16 ***
PULocationID_encoded        -0.0063700  0.0002046 -31.134  < 2e-16 ***
DOLocationID_encoded         0.0267073  0.0001765 151.332  < 2e-16 ***
VendorID_encoded            -0.0933332  0.0070811 -13.181  < 2e-16 ***
factor(store_and_fwd_flag)Y -0.0403986  0.0073180  -5.520 3.38e-08 ***
factor(payment_type)2        0.1780888  0.0024170  73.681  < 2e-16 ***
factor(payment_type)3        0.0028548  0.0073928   0.386  